In [14]:
from utils.algorithm import overlay_of_subdivision


w = [(1,1), (2,2), (1,3), (0,2)]
w_e = [[i, i+1] for i in range(len(w)-1)]
w_e.append([len(w) - 1, 0])
v = [(x[0] + 1, x[1]) for x in w]
v_e = [[i, i+1] for i in range(len(v)-1)]
v_e.append([len(v) - 1, 0])

import matplotlib.pyplot as plt

def plot_fig(a, color):
    for l in a:
        l_x = [x[0] for x in l]
        l_y = [y[1] for y in l]
        plt.plot(l_x, l_y, marker='s', color=color)
        
S1 = DCEL(w, w_e)
S2 = DCEL(v, v_e)
S = overlay_of_subdivision(S1, S2)

KeyError: (1e-10, 2.0000000001)